In [66]:
from PIL import Image,ImageFile
import io
import os
import pandas as pd
import csv
import sys
import cv2
import numpy as np
import time
size1=640
size2=360
import torch
from torchvision.transforms import ColorJitter, ToTensor, Normalize,Compose,Resize,ToPILImage
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from PIL import Image, ImageOps, ImageEnhance
import numbers
device = "cpu"
def pic_preprocessing(ar):
    if ar.shape[0]==1024:
        ar=ar[300:,450:1350]
        ar[:350,:350]=0
    else:
        ar=ar[:310,180:420]
        ar[:160,:80]=0
    return Image.fromarray(ar)
class DatasetLoader(Dataset):
    def __init__(self,root_dir=r"D:\Projects\Order_counting\regression",transforms=None,mode="train"):
        self.root_dir=root_dir
        self.mode=mode
        self.transforms=transforms
        if mode=="train":
            self.data=os.listdir(root_dir+"\\train")
            self.label=[name.split('.')[0].split('_')[1] for name in self.data]
        elif mode=="val":
            self.data=os.listdir(root_dir+"\\test")
            self.label=[name.split('.')[0].split('_')[1] for name in self.data]
        else:
            self.data=os.listdir(root_dir+"\\test")
#            self.label=[name.split('.')[0].split('_')[1] for name in data]
    def __len__(self):
        return len(self.data)
    def __getitem__(self,idx):
        if self.mode=='train':
            file_name=self.root_dir+"\\train\\"+self.data[idx]
            img=Image.open(file_name)
            img=pic_preprocessing(np.array(img))
            if self.transforms:
                img=self.transforms(img)
            image_label=self.label[idx]
            image_label=torch.tensor(int(image_label),dtype=torch.int64)
            return (img,image_label)
        elif self.mode=='val':
            file_name=self.root_dir+"\\test\\"+self.data[idx]
            img=Image.open(file_name)
            img=pic_preprocessing(np.array(img))
            if self.transforms:
                img=self.transforms(img)
            image_label=self.label[idx]
            image_label=torch.tensor(int(image_label),dtype=torch.int64)
            return (img,image_label)
        else:
            file_name=self.root_dir+"\test\\"+self.data[idx]
            img=Image.open(file_name)
            img=pic_preprocessing(np.array(img))
            if self.transforms:
                img=self.transforms(img)
            return (img)
# import Augmentor        
# p=Augmentor.Pipeline()
# p.zoom(probability=0.4, min_factor=1.1, max_factor=1.5)
# p.crop_centre(probability=0.4,percentage_area=0.2)
# p.random_brightness(probability=0.4,min_factor=0.7, max_factor=1.5)
# p.random_color(probability=0.4, min_factor=0.5, max_factor=2)
# p.gaussian_distortion(probability=0.2, grid_width=5, grid_height=5, magnitude=5,corner='bell',method='in')

train_transform = Compose([
#    p.torch_transform(),
#    ToPILImage(),
    Resize((size2,size1)),
#    RandomRotation(degrees=10),
#    RandomShift(4),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406],
std=[0.229, 0.224, 0.225])
])
val_transform = Compose([
    Resize((size2,size1)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406],
std=[0.229, 0.224, 0.225])
])
batch_size = 16

train_dataset = DatasetLoader(transforms=train_transform)
test_dataset = DatasetLoader(transforms=train_transform,mode="test")
val_dataset = DatasetLoader(transforms=train_transform,mode="val")

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, 
                          drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, 
                          drop_last=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True,
                        drop_last=False)

import torch.nn as nn
import torchvision.models as models
model=models.resnext101_32x8d(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
# for param in model.features.denseblock4.parameters():
#     param.requires_grad=True
for param in model.layer4.parameters():
    param.requires_grad=True
num_ftrs = model.fc.in_features
# model.classifier = nn.Sequential(nn.BatchNorm1d(num_ftrs),
#                                  nn.Dropout(0.25),
#                                 )
model.fc = nn.Linear(num_ftrs,8)                                
model = model.to(device)
#weights = [0.065, 1]
#class_weights=torch.FloatTensor(weights)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [67]:
epochs = 30
steps = 0
running_loss = 0
print_every = 10
train_losses, test_losses = [], []
train_accuracy=0
best_train=0
best_val=0

for epoch in range(epochs):
    for inputs, labels in train_loader:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        train_accuracy +=torch.mean(equals.type(torch.FloatTensor)).item()
        
        if steps % print_every == 0:
            test_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device),labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    test_loss += batch_loss.item()
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy +=torch.mean(equals.type(torch.FloatTensor)).item()
                    
            train_losses.append(running_loss/len(train_loader))
            test_losses.append(test_loss/len(val_loader))                    
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Train accuracy: {train_accuracy/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(val_loader):.3f}.. "
                  f"Test accuracy: {accuracy/len(val_loader):.3f}")
            if accuracy>=best_val:
                best_val=accuracy
                torch.save(model,f'D:\Projects\order_counting\models\\resnext_model_train-{train_accuracy/print_every:.3f}_test-{accuracy/len(val_loader):.3f}.pt')
                print('Model saved!')
            running_loss = 0
            train_accuracy=0
            model.train()

Epoch 1/30.. Train loss: 2.030.. Train accuracy: 0.225.. Test loss: 12.235.. Test accuracy: 0.243


C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ResNet. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type BatchNorm2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
C:\Users\ToschevikovG.EUROOPT\AppData\Local\Continuum\anaconda3\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve

Model saved!
Epoch 2/30.. Train loss: 1.591.. Train accuracy: 0.356.. Test loss: 3.461.. Test accuracy: 0.108
Epoch 2/30.. Train loss: 1.681.. Train accuracy: 0.244.. Test loss: 1.775.. Test accuracy: 0.274
Model saved!
Epoch 3/30.. Train loss: 1.683.. Train accuracy: 0.312.. Test loss: 1.615.. Test accuracy: 0.431
Model saved!
Epoch 3/30.. Train loss: 1.553.. Train accuracy: 0.375.. Test loss: 1.445.. Test accuracy: 0.403
Epoch 4/30.. Train loss: 1.469.. Train accuracy: 0.450.. Test loss: 1.306.. Test accuracy: 0.451
Model saved!
Epoch 5/30.. Train loss: 1.452.. Train accuracy: 0.425.. Test loss: 2.044.. Test accuracy: 0.319
Epoch 5/30.. Train loss: 1.183.. Train accuracy: 0.588.. Test loss: 1.786.. Test accuracy: 0.451
Model saved!
Epoch 6/30.. Train loss: 0.998.. Train accuracy: 0.688.. Test loss: 2.873.. Test accuracy: 0.212
Epoch 6/30.. Train loss: 0.863.. Train accuracy: 0.675.. Test loss: 7.092.. Test accuracy: 0.285
Epoch 7/30.. Train loss: 0.664.. Train accuracy: 0.744.. Test 

KeyboardInterrupt: 

In [45]:
model = torch.load('D:\Projects\Order_counting\models\densenet_model_train-1.000_test-1.000.pt')
model.eval()
test_transforms = Compose([Resize((size2,size1)),
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406],
std=[0.229, 0.224, 0.225])
])
img=Image.open(r"D:\Projects\Order_counting\regression\test\678image_6.jpg")
img=pic_preprocessing(np.array(img))
img=test_transforms(img).float()
img=img.unsqueeze_(0)
model(img)



tensor([[-6.4652, -2.6306, -1.8200,  1.6951,  3.7296, -0.1632,  2.1374]],
       grad_fn=<AddmmBackward>)

In [65]:
model.layer4

Sequential(
  (0): Bottleneck(
    (conv1): Conv2d(1024, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(2048, 2048, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=32, bias=False)
    (bn2): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv3): Conv2d(2048, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (downsample): Sequential(
      (0): Conv2d(1024, 2048, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): Bottleneck(
    (conv1): Conv2d(2048, 2048, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=